In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mjol import gan as mgan

In [3]:
fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/data/chm13v2.0_RefSeq_Liftoff_v5.2.gff3"

In [4]:
gan = mgan.GAn(
    filename = fn,
    format = 'gff'
)

In [ ]:
gan.build_db(use_iv = False)

exon
cds


In [4]:
import pandas as pd

df = pd.read_csv(fn, sep='\t', comment='#', header=None)
df.columns = mgan.HDR
df.head()

,chr,src,feature_type,start,end,score,strand,frame,attributes
0,chr1,Liftoff,gene,6047,13941,.,-,.,ID=LOC124900618;gene_name=LOC124900618;db_xref...
1,chr1,Liftoff,transcript,6047,13941,.,-,.,ID=XR_002958507.2;Parent=LOC124900618;db_xref=...
2,chr1,Liftoff,exon,6047,6420,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
3,chr1,Liftoff,exon,12078,12982,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
4,chr1,Liftoff,exon,13445,13579,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...


In [5]:
from mjol import base as mbase

In [6]:
tx = None
for _, row in df.iterrows():
    if row['feature_type'] == "transcript":
        if tx is not None:
            break
        tx = mbase.IntervalGFeature(
                chr = row['chr'],
                src = row['src'],
                feature_type = row['feature_type'],
                start = row['start'],
                end = row['end'],
                score = None if row['score'] == '.' else row['score'],
                strand = row['strand'],
                frame = row['frame'],
                attributes = mbase.load_attributes(row['attributes'], 
                                        kv_sep = '='),
                children = []
            )
    elif row['feature_type'] == "exon":
        exon = mbase.GFeature(
                chr = row['chr'],
                src = row['src'],
                feature_type = row['feature_type'],
                start = row['start'],
                end = row['end'],
                score = None if row['score'] == '.' else row['score'],
                strand = row['strand'],
                frame = row['frame'],
                attributes = mbase.load_attributes(row['attributes'], 
                                        kv_sep = '='),
                children = []
            )
        tx.add_a_child(exon)


In [19]:
tx.build_itree_from_children()

In [7]:
tx.query_itree(12100, 12200)

{Interval(12078, 12982, exon:Supporting evidence includes similarity to: 1 long SRA read%2C and 90%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1 sample with support for all annotated introns,-,12078-12982)}